In [1]:

#Libraries
import numpy as np
import itertools
import os
import random
import tensorflow as tf
from cleverhans.attacks import FastGradientMethod
from io import BytesIO
import IPython.display
import numpy as np
import pandas as pd
from PIL import Image
from scipy.misc import imread
from scipy.misc import imsave
import tensorflow as tf
# from tensorflow.contrib.slim.nets import inception

from config import Config

# Defenses
from jpeg_denoise import jpeg
from fourier_whole_denoise import fourier_whole, get_subdirs
from pca_whole_denoise import pca_whole
from pca_blockwise_denoise import pca_blockwise


In [ ]:
config           = Config()
dataset_path     = "dataset/"
attack_base_dir  = dataset_path + "attack/"
denoise_base_dir = dataset_path + "DenoisedSamples/"
benign_dir        = dataset_path + "attack/benign"

In [ ]:
jpeg_denoise.jpeg(attack_base_dir, denoise_base_dir, 75)

In [ ]:
fourier_whole(attack_base_dir, denoise_base_dir, 65, 65)

In [ ]:
pca_whole(attack_base_dir, denoise_base_dir, 36)

In [ ]:
pca_blockwise(attack_base_dir, denoise_base_dir, 2)

In [ ]:
pca_blockwise(attack_base_dir, denoise_base_dir, )

In [ ]:
def get_image_list(benign_dir, random_seed, subset_num):
    images_whole      = [f for f in os.listdir(benign_dir) if os.path.isfile(os.path.join(benign_dir, f))]
    images_whole      = sorted(images_whole)

    if random_seed != None:
        random.seed(random_seed)
        subset_ids  = random.sample(range(len(images_whole)), subset_num)  # generate random indices to subset
    else:
        subset_ids  = range(subset_num)

    image_list = [images_whole[i] for i in subset_ids]
    return image_list
def load_images(input_dir, batch_shape):
    images = np.zeros(batch_shape)
    filenames = []
    idx = 0
    batch_size = batch_shape[0]
    for filepath in sorted(tf.gfile.Glob(os.path.join(input_dir, '*.png'))):
        with tf.gfile.Open(filepath, "rb") as f:
            images[idx, :, :, :] = imread(f, mode='RGB').astype(np.float)*2.0/255.0 - 1.0
        filenames.append(os.path.basename(filepath))
        idx += 1
        if idx == batch_size:
            yield filenames, images
            filenames = []
            images = np.zeros(batch_shape)
            idx = 0
    if idx > 0:
        yield filenames, images
        
def show_image(a, fmt='png'):
    a = np.uint8((a+1.0)/2.0*255.0)
    f = BytesIO()
    Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [ ]:
image_iterator = load_images(benign_dir, config.batch_shape)

In [ ]:
filenames, images = next(image_iterator)
show_image(images[0])

In [ ]:
show_image(images[12])

In [ ]:
im_list = list(image_iterator);
len(im_list)